In [1]:
import pandas as pd
import os
import joblib
import numpy as np

In [2]:
if_debug = False
KFOLD = 5
# sample upload file path
sample_upload_file_path1 = r"E:\新桌面文件存放位置\招商银行\数据集\科创企业行业先进性判断子模型\upload.csv"
sample_upload_file_path2 = r"E:\新桌面文件存放位置\招商银行\数据集\科创企业成长潜力模型\upload.csv"

upload_file1 = pd.read_csv(sample_upload_file_path1)
upload_file2 = pd.read_csv(sample_upload_file_path2)

In [3]:
upload_fold_name = '07012032_try' # 本次提交的输出文件夹路径
upload_path = os.path.join('./upload_file', upload_fold_name)
if not os.path.exists(upload_path):
    os.makedirs(upload_path)

In [4]:
save_model_dir_path = r'./save_models/0701_1930'
with open(os.path.join(save_model_dir_path, 'threshold.txt'), 'r') as f:
    line = f.readline()
    ths = line.strip().split(' ')
    ths = [float(x) for x in ths]
    best_ths = sum(ths) / KFOLD
best_ths

0.29

In [5]:
# upload_file1

In [6]:
# upload_file2

In [7]:
if if_debug:
    # 测试提交
    upload_file1['score'] = 0
    upload_file1['Y'] = 0
    upload_file2['score'] = 0
    upload_file2['Y'] = 0
else:
    # test_data_path1 = ''

    # 任务二提交
    upload_file1['score'] = 0
    upload_file1['Y'] = 0
    test_data_path2 = r'./processed_data/all_data_1930.csv'
    all_data = pd.read_csv(test_data_path2)
    all_features = list(all_data.columns)
    all_features = [x for x in all_features if x not in ['Unnamed: 0', 'ID', 'score','Y',]]

    all_test_data = all_data[all_data['Y']==-1]
    dftest_X = all_test_data[all_features]
    dftest_X = dftest_X.fillna(0) # 填充0

    # 加载模型
    oof_pred = []
    for i in range(KFOLD):
        lgbm_loaded = joblib.load(os.path.join(save_model_dir_path, f'model_{i}.pkl'))
        each_fold_predict = lgbm_loaded.predict_proba(dftest_X)[:,1]
        oof_pred.append(each_fold_predict)
        # all_test_data['pred_score'] = lgbm_loaded.predict_proba(dftest_X)[:,1]
        # print('y_prob:', y_prob)
        # upload_file2 = upload_file2.merge(all_test_data[['ID', 'pred_score']], on='ID', how="left")


In [8]:
pred_all = np.sum(np.array(oof_pred), axis=0)/KFOLD
all_test_data['pred_score'] = pred_all

C:\Users\HP\AppData\Local\Temp\ipykernel_30604\2075056530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_test_data['pred_score'] = pred_all


In [9]:
upload_file2 = upload_file2.merge(all_test_data[['ID', 'pred_score']], on='ID', how="left")
upload_file2 = upload_file2[['ID', 'pred_score']]
upload_file2.rename(columns={'pred_score':'score'}, inplace=True)
upload_file2['Y'] = 1
upload_file2.loc[upload_file2['score'] < 0.5, 'Y'] = 0

In [10]:
# 保存数据集
upload_file1_path = os.path.join(upload_path, 'upload1.csv')
upload_file2_path = os.path.join(upload_path, 'upload2.csv')

upload_file1.to_csv(upload_file1_path, encoding='utf-8-sig', index=False)
upload_file2.to_csv(upload_file2_path, encoding='utf-8-sig', index=False)

In [11]:
upload_file2['Y'].sum()

197